In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('high_diamond_ranked_10min.csv')
df

,gameId,blueWins,blueWardsPlaced,blueWardsDestroyed,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,...,redTowersDestroyed,redTotalGold,redAvgLevel,redTotalExperience,redTotalMinionsKilled,redTotalJungleMinionsKilled,redGoldDiff,redExperienceDiff,redCSPerMin,redGoldPerMin
0,4519157822,0,28,2,1,9,6,11,0,0,...,0,16567,6.8,17047,197,55,-643,8,19.7,1656.7
1,4523371949,0,12,1,0,5,5,5,0,0,...,1,17620,6.8,17438,240,52,2908,1173,24.0,1762.0
2,4521474530,0,15,0,0,7,11,4,1,1,...,0,17285,6.8,17254,203,28,1172,1033,20.3,1728.5
3,4524384067,0,43,1,0,4,5,5,1,0,...,0,16478,7.0,17961,235,47,1321,7,23.5,1647.8
4,4436033771,0,75,4,0,6,6,6,0,0,...,0,17404,7.0,18313,225,67,1004,-230,22.5,1740.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9874,4527873286,1,17,2,1,7,4,5,1,1,...,0,15246,6.8,16498,229,34,-2519,-2469,22.9,1524.6
9875,4527797466,1,54,0,0,6,4,8,1,1,...,0,15456,7.0,18367,206,56,-782,-888,20.6,1545.6
9876,4527713716,0,23,1,0,6,7,5,0,0,...,0,18319,7.4,19909,261,60,2416,1877,26.1,1831.9
9877,4527628313,0,14,4,1,2,3,3,1,1,...,0,15298,7.2,18314,247,40,839,1085,24.7,1529.8


## Verificare daca exista valori NaN

In [ ]:
df.isna().any().any()

## Verificare matrice de corelatie
### Heatmap

In [ ]:
correlation_matrix = df.corr()

# Afișează matricea de corelație
correlation_matrix
plt.figure(figsize=(35, 25))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Matricea de corelație')
plt.show()


## Verificare daca exista corelatii cu valori mai mari de 0.95

In [ ]:
corr_unstack = correlation_matrix.unstack().reset_index()
corr_unstack.columns = ['var1', 'var2', 'correlation']

# Filtrează corelațiile mai mari de 0.95 și elimină auto-corelațiile (variabilă cu ea însăși)
high_corr = corr_unstack[(abs(corr_unstack['correlation']) > 0.95) & 
                         (corr_unstack['var1'] != corr_unstack['var2'])]

# Elimină duplicate (A-B și B-A reprezintă aceeași corelație)
high_corr = high_corr[high_corr['var1'] < high_corr['var2']]

# Sortează rezultatele după valoarea corelației (descrescător)
high_corr = high_corr.sort_values('correlation', ascending=False)

# Afișează rezultatele
high_corr


## Eliminarea corelatiilor mai mari de 0.95 , pastrarea variabilei Redkills in schimbul BlueDeaths

In [ ]:
# Creează o listă de coloane de eliminat bazată pe corelațiile mari
columns_to_drop = []

# Parcurge toate perechile de variabile puternic corelate
for _, row in high_corr.iterrows():
    var1, var2 = row['var1'], row['var2']
    
    # Cazul specific: dacă perechea conține blueDeaths și redKills
    if ('blueDeaths' in [var1, var2] and 'redKills' in [var1, var2]):
        columns_to_drop.append('blueDeaths')  # Elimină întotdeauna blueDeaths
    else:
        # Altfel, păstrează comportamentul original
        columns_to_drop.append(var2)

# Elimină duplicate din lista de coloane (dacă există)
columns_to_drop = list(set(columns_to_drop))

# Creează un nou DataFrame fără variabilele puternic corelate
df = df.drop(columns=columns_to_drop)

# Afișează câte coloane au fost eliminate
print(f"Au fost eliminate {len(columns_to_drop)} coloane din {df.shape[1]} (rămase {df.shape[1]}):")
print(columns_to_drop)

# Afișează noul DataFrame
df.head()


## Verificare daca exista duplicate

In [ ]:

duplicate_count = df.duplicated().sum()

# Afișează rezultatul
print(f"Număr de rânduri duplicate: {duplicate_count}")

# Dacă există duplicate, afișează-le
if duplicate_count > 0:
    print("\nExemple de rânduri duplicate:")
    duplicates = df[df.duplicated(keep='first')]
    display(duplicates.head())
    
    # Opțional: Afișează toate duplicatele cu rândurile originale
    print("\nToate rândurile duplicate și originalele lor:")
    all_duplicates = df[df.duplicated(keep=False)]
    display(all_duplicates.sort_values(by=all_duplicates.columns.tolist()))


##  Verificare daca exista valori NULL

In [ ]:
has_null = df.isnull().values.any()
print(f"DataFrame-ul conține valori null: {has_null}")

# Numărul total de valori null
total_null = df.isnull().sum().sum()
print(f"Numărul total de valori null: {total_null}")


## Schimbare game_id intr-o valoare mai mica, aceasta fiind deja o valoare unica

In [ ]:
# First, check the current range of game_id values
print(f"Current min game_id: {df['gameId'].min()}")
print(f"Current max game_id: {df['gameId'].max()}")
print(f"Number of unique games: {df['gameId'].nunique()}")

# Create a mapping of old IDs to new IDs
unique_ids = df['gameId'].unique()
id_mapping = {old_id: new_id + 1 for new_id, old_id in enumerate(unique_ids)}

# Create a new column with remapped IDs
df['game_id_new'] = df['gameId'].map(id_mapping)

# Verify the new range
print(f"New min game_id: {df['game_id_new'].min()}")
print(f"New max game_id: {df['game_id_new'].max()}")

# If you want to replace the old IDs completely
# df['gameId'] = df['game_id_new']
# df.drop(columns=['game_id_new'], inplace=True)

# Replace the original gameId with the remapped values
df['gameId'] = df['game_id_new']
df.drop(columns=['game_id_new'], inplace=True)


# Preview the updated DataFrame
df


## Histograma


In [ ]:

# Selectează doar coloanele numerice
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
num_cols = len(numeric_cols)

# Calculează numărul de rânduri și coloane pentru subplot grid
import math
n_rows = math.ceil(num_cols / 3)

# Creează histograme pentru toate coloanele numerice
fig, axes = plt.subplots(n_rows, 3, figsize=(18, n_rows*4))
axes = axes.flatten()

for i, col in enumerate(numeric_cols):
    sns.histplot(df[col], kde=True, ax=axes[i], color='skyblue', edgecolor='black')
    axes[i].set_title(f'Distribuția {col}')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Frecvență')
    
# Ascunde axele rămase nefolosite (dacă există)
for j in range(i+1, len(axes)):
    axes[j].set_visible(False)
    
plt.tight_layout()
plt.show()


## Heatmap

In [ ]:
plt.figure(figsize=(35, 25))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt=".2f", linewidths=1)
plt.title('Matricea de corelație')
plt.show()


Describe